# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

# Read the Dataset

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='1XtFra1TyfI7hJVndDZaCNVcW7HXvVY_3GaJTeIIGdd3',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'carresalevalueprediction-donotdelete-pr-2qsbloaqlvccm1'
object_key = 'autos_preprocessed.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()


Unnamed: 0  price vehicleType  yearOfRegistration    gearbox  powerPS  \
0           1  18300       coupe                2011     manual      190   
1           2   9800         suv                2004  automatic      163   
2           3   1500   small car                2001     manual       75   
3           4   3600   small car                2008     manual       69   
4           5    650   limousine                1995     manual      102   

          model  kilometer  monthOfRegistration fuelType       brand  \
0  not-declared     125000                    5   diesel        audi   
1         grand     125000                    8   diesel        jeep   
2          golf     150000                    6   petrol  volkswagen   
3         fabia      90000                    7   diesel       skoda   
4           3er     150000                   10   petrol         bmw   

  notRepairedDamage  
0               Yes  
1      not-declared  
2                No  
3                No  
4               Yes

In [4]:
data.tail()

Unnamed: 0  price  vehicleType  yearOfRegistration    gearbox  \
278573      371520   3200    limousine                2004     manual   
278574      371524   1199  convertible                2000  automatic   
278575      371525   9200          bus                1996     manual   
278576      371526   3400  combination                2002     manual   
278577      371527  28990    limousine                2013     manual   

        powerPS        model  kilometer  monthOfRegistration fuelType  \
278573      225         leon     150000                    5   petrol   
278574      101       fortwo     125000                    3   petrol   
278575      102  transporter     150000                    3   diesel   
278576      100         golf     150000                    6   diesel   
278577      320      m_reihe      50000                    8   petrol   

             brand notRepairedDamage  
278573        seat               Yes  
278574       smart                No  
278575  volkswagen                No  
278576  volkswagen      not-declared  
278577         bmw                No

# Label encoding the categorical data

In [5]:
labels = ['gearbox', 'notRepairedDamage', 'model', 'brand', 'fuelType', 'vehicleType']

mapper = {}
for i in labels:
    mapper[i] = LabelEncoder()
    mapper[i].fit(data[i])
    tr = mapper[i].transform(data[i])
    np.save(str('classes'+i+'.npy'), mapper[i].classes_)
    data.loc[:, i+'_labels'] = pd.Series(tr, index=data.index)
    
labeled = data[['price', 'yearOfRegistration','powerPS','kilometer','monthOfRegistration']
                  +[x+"_labels" for x in labels]]

print(labeled.columns)

Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278578 entries, 0 to 278577
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   Unnamed: 0                278578 non-null  int64 
 1   price                     278578 non-null  int64 
 2   vehicleType               278578 non-null  object
 3   yearOfRegistration        278578 non-null  int64 
 4   gearbox                   278578 non-null  object
 5   powerPS                   278578 non-null  int64 
 6   model                     278578 non-null  object
 7   kilometer                 278578 non-null  int64 
 8   monthOfRegistration       278578 non-null  int64 
 9   fuelType                  278578 non-null  object
 10  brand                     278578 non-null  object
 11  notRepairedDamage         278578 non-null  object
 12  gearbox_labels            278578 non-null  int64 
 13  notRepairedDamage_labels  278578 non-null  int64 
 14  mode

# Splitting Data Into Independent And Dependent Variables

In [7]:
Y = labeled.iloc[:,0].values

In [8]:
X= labeled.iloc[:,1:].values

In [9]:
Y=Y.reshape(-1,1)

In [10]:
from sklearn.model_selection import cross_val_score, train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state = 3)

# Choose The Appropriate Model

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score


In [12]:
regressor = RandomForestRegressor(n_estimators=1000,max_depth=10,random_state=34)

In [13]:
regressor.fit(X_train, np.ravel(Y_train,order='C'))

RandomForestRegressor(max_depth=10, n_estimators=1000, random_state=34)

# Check The Metrics Of The Model

In [14]:
y_pred = regressor.predict(X_test)

In [15]:
r2=r2_score(Y_test,y_pred)
print("R2_score:",r2)

R2_score: 0.834527626497731


In [16]:
Adjusted_R2=1-(1-r2*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))
print("Adjusted R2:",Adjusted_R2)

Adjusted R2: 0.8346274945764857


In [17]:
from sklearn.metrics import mean_squared_error
import math

In [18]:
MSE=mean_squared_error(Y_test,y_pred)
print("MSE:",MSE)

MSE: 11837192.971239958


In [19]:
RMSE=math.sqrt(MSE)
print("RMSE:",RMSE)

RMSE: 3440.5221945570934


In [20]:
plt.figure(figsize=(10,5))
plt.plot(y_pred[0:20])
plt.plot(np.array(Y_test[0:20]))
plt.legend(["predicted","actual"])
plt.show()

# Save the model

In [21]:
filename='resale_model.sav'
pickle.dump(regressor,open(filename,'wb'))

# Deployment

In [22]:
!pip install ibm_watson_machine_learning

In [23]:
from ibm_watson_machine_learning import APIClient
import json
import numpy as np

In [37]:
wml_credentials = {
    "apikey" : "PWwXehQUinqBMRkX5mCDI0T8ZqSTsnmvY65vzYazuK1t",
    "url" : "https://us-south.ml.cloud.ibm.com" #For Dallas region
}

In [38]:
client =APIClient(wml_credentials)

In [39]:
def guid_from_space_name(Client,space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [40]:
space_uid = guid_from_space_name(client, 'models')
print("Space UID ="+ space_uid )

Space UID =602cf1de-504e-4aab-a0f3-ec85de6ed13b


In [41]:
client.set.default_space(space_uid)

'SUCCESS'

In [42]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [43]:
software_spec_uid = client.software_specifications.get_uid_by_name('runtime-22.1-py3.9')
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [44]:
import sklearn
sklearn.__version__

'1.0.2'

# Create Model Properties to deploy the model

In [46]:
model_details= client.repository.store_model(model= regressor,meta_props={
    client.repository.ModelMetaNames.NAME: "Prediction_model",
    client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid}
                                            )
model_id = client.repository.get_model_id(model_details)

In [47]:
model_id

'30d6c074-1f4b-45b5-a311-04fff5de3f42'